In [1]:
%cd ..
import torch
from meldataset import build_dataloader
import yaml

from utils import *
from models import *


c:\Users\catto\Desktop\Code\Project\TTS\StyleTTS2


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = yaml.safe_load(open('Configs/config.yml'))

In [3]:
def get_data_path_list(train_path=None, val_path=None):
    if train_path is None:
        train_path = "Data/train_list.txt"
    if val_path is None:
        val_path = "Data/val_list.txt"

    with open(train_path, 'r', encoding='utf-8', errors='ignore') as f:
        train_list = f.readlines()
    with open(val_path, 'r', encoding='utf-8', errors='ignore') as f:
        val_list = f.readlines()

    return train_list, val_list

In [4]:
train_path = "../../Data_Speech/LibriTTS/train.txt"
val_path = "../../Data_Speech/LibriTTS/val.txt"
train_list, val_list = get_data_path_list(train_path, val_path)

root_path = "../../Data_Speech/"
min_length = 50
batch_size = 3

In [5]:
train_dataloader = build_dataloader(train_list,
                                    root_path,
                                    min_length=min_length,
                                    batch_size=batch_size,
                                    num_workers=2,
                                    dataset_config={},
                                    device=device)

In [6]:
model_params = recursive_munch(config['model_params'])
model = build_model(model_params)
_ = [model[key].to(device) for key in model]

c:\Users\catto\anaconda3\Lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
c:\Users\catto\anaconda3\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [11]:
model.keys()

dict_keys(['decoder', 'predictor', 'text_encoder', 'predictor_encoder', 'style_encoder', 'text_aligner', 'pitch_extractor', 'mpd', 'msd'])

In [9]:
model_int8 = torch.ao.quantization.quantize_dynamic(
    model.decoder,  # the original model
    {torch.nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.qint8)

RuntimeError: Only Tensors created explicitly by the user (graph leaves) support the deepcopy protocol at the moment.  If you were attempting to deepcopy a module, this may be because of a torch.nn.utils.weight_norm usage, see https://github.com/pytorch/pytorch/pull/103001